<a href="https://colab.research.google.com/github/Master-Shuvam/Dhanvantari1/blob/master/Dhanvantari_phase1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("DiseaseAndSymptoms.csv")

# Fill NaNs with empty string
df = df.fillna('')

# Combine all symptom columns into one text column
symptom_cols = [col for col in df.columns if 'Symptom' in col]
df['symptoms_text'] = df[symptom_cols].apply(lambda x: ', '.join([i.strip() for i in x if i != '']), axis=1)

# Keep only Disease and combined text
df_final = df[['disease_label', 'symptoms_text']]

# Preview
print(df_final.head())


       disease_label symptoms_text
0     Pyelonephritis              
1  Stroke (ischemic)              
2   Tension Headache              
3               Gout              
4               Gout              


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Convert Disease column to numbers
df_final['label'] = le.fit_transform(df_final['disease_label'])

# Check mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)


{'Absence Seizures': np.int64(0), 'Acute Appendicitis': np.int64(1), 'Allergic Rhinitis': np.int64(2), 'Alzheimer Disease': np.int64(3), 'Amyotrophic Lateral Sclerosis (ALS)': np.int64(4), 'Angina': np.int64(5), 'Aortic Aneurysm': np.int64(6), 'Aortic Dissection': np.int64(7), 'Aortic Regurgitation': np.int64(8), 'Aortic Stenosis': np.int64(9), 'Arrhythmogenic Right Ventricular Cardiomyopathy': np.int64(10), 'Asthma': np.int64(11), 'Astrocytoma': np.int64(12), 'Ataxia': np.int64(13), 'Atrial Fibrillation': np.int64(14), 'Atrial Septal Defect': np.int64(15), 'Bell?? Palsy': np.int64(16), 'Benign Prostatic Hyperplasia': np.int64(17), 'Bradyarrhythmia': np.int64(18), 'Brain Abscess': np.int64(19), 'Bronchitis': np.int64(20), 'COPD': np.int64(21), 'COVID-19': np.int64(22), 'CTEPH': np.int64(23), 'Cardiac Amyloidosis': np.int64(24), 'Carpal Tunnel Syndrome': np.int64(25), 'Cellulitis': np.int64(26), 'Cerebral Aneurysm': np.int64(27), 'Cerebral Palsy': np.int64(28), 'Coarctation of Aorta': n

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")


Training samples: 5366, Validation samples: 1342


In [ ]:
from transformers import AutoTokenizer

# Load ClinicalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Tokenize training data
train_encodings = tokenizer(list(train_df['symptoms_text']),
                            truncation=True,
                            padding=True,
                            max_length=128)

val_encodings = tokenizer(list(val_df['symptoms_text']),
                          truncation=True,
                          padding=True,
                          max_length=128)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
import torch

train_labels = torch.tensor(list(train_df['label']))
val_labels = torch.tensor(list(val_df['label']))


In [ ]:
class SymptomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = SymptomDataset(train_encodings, train_labels)
val_dataset = SymptomDataset(val_encodings, val_labels)


In [ ]:
pip install transformers --upgrade

7

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Number of disease classes
num_labels = len(le.classes_)

# Load ClinicalBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=num_labels
)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


8

In [ ]:
# DISABLE WANDB LOGGING (to avoid API key prompt)
import os
os.environ["WANDB_DISABLED"] = "true"

# First, let's check your transformers version
import transformers
print(f"Transformers version: {transformers.__version__}")

# Step 7: Define Training Arguments (MINIMAL VERSION - SHOULD WORK)
from transformers import TrainingArguments

# Minimal configuration that should work with any version
training_args = TrainingArguments(
    output_dir="./results",              # where to save model checkpoints
    learning_rate=1e-5,                  # standard for BERT fine-tuning
    per_device_train_batch_size=8,       # reduced batch size for compatibility
    num_train_epochs=10,                  # number of training epochs
    logging_steps=10,                    # log every 10 steps
    save_steps=500,                      # save checkpoint every 500 steps
    report_to="none",                    # Disable all logging services
)

# If you want evaluation during training, try this after seeing your version:
print("If you want evaluation, try one of these based on your version:")
print("- For newer versions (4.0+): eval_strategy='epoch'")
print("- For older versions (3.x): evaluation_strategy='epoch'")
print("- For very old versions (2.x): evaluate_during_training=True")

# Let's create a version that definitely works first
print("Using minimal training args that should work with any version...")

# Step 8: Create Trainer Object
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)




Transformers version: 4.56.1
If you want evaluation, try one of these based on your version:
- For newer versions (4.0+): eval_strategy='epoch'
- For older versions (3.x): evaluation_strategy='epoch'
- For very old versions (2.x): evaluate_during_training=True
Using minimal training args that should work with any version...


9

In [ ]:
print("Starting training...")
trainer.train()

Starting training...


Step,Training Loss
10,4.798000
20,4.870000
30,4.960100
40,4.827800
50,4.910300
60,4.804500
70,4.819700
80,4.759400
90,4.732800
100,4.695500


TrainOutput(global_step=6710, training_loss=4.332484151780516, metrics={'train_runtime': 595.8152, 'train_samples_per_second': 90.061, 'train_steps_per_second': 11.262, 'total_flos': 55214916552000.0, 'train_loss': 4.332484151780516, 'epoch': 10.0})

10

In [ ]:
print("Saving model...")
model.save_pretrained("./clinicalbert_symptom_model")
tokenizer.save_pretrained("./clinicalbert_symptom_model")

print("Training completed and model saved!")

Saving model...
Training completed and model saved!


11

In [ ]:
from google.colab import files
import shutil

# 1️⃣ Compress the fine-tuned model folder into a ZIP
shutil.make_archive("clinicalbert_symptom_model", 'zip', "./clinicalbert_symptom_model")

# 2️⃣ Download the ZIP file to your computer
files.download("clinicalbert_symptom_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

files.download("label_encoder.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

12

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the model & tokenizer from the folder
model_path = "./clinicalbert_symptom_model"  # adjust path if needed
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Put model in evaluation mode
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Example symptom input
symptom_text = "I have cough,running nose"

# Tokenize
inputs = tokenizer(
    symptom_text,
    return_tensors="pt",      # return PyTorch tensors
    padding=True,
    truncation=True,
    max_length=128
)


In [ ]:
# Get logits (raw outputs)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get predicted class
predicted_class_id = torch.argmax(logits, dim=1).item()


In [ ]:
# Step 4: Convert Class ID to Disease Name using LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Recreate the LabelEncoder from your training data
le = LabelEncoder()
le.fit(df_final['Disease'])  # df_final['Disease'] is your training disease column

# Convert predicted class ID to disease name
predicted_disease = le.inverse_transform([predicted_class_id])[0]
print("Predicted Disease:", predicted_disease)


Predicted Disease: GERD


from here seperate code. the training code does not include this part


In [ ]:
import torch.nn.functional as F


In [ ]:
disease_symptoms = {
    "Hypertension (High BP)": ["headache", "dizziness", "nosebleeds", "blurred vision"],
    "Low BP": ["dizziness", "fainting", "blurred vision", "fatigue"],
    "Heart Attack (Myocardial Infarction)": ["chest pain", "shortness of breath", "sweating", "nausea","arm pain","palpitations"],
    "Stroke": ["fatigue","sudden weakness", "difficulty speaking", "blurred vision", "loss of balance"],
    "Asthma": ["wheezing", "shortness of breath", "chest tightness", "cough"],
    "Pneumonia": ["fast breathing","shallow breathing","fever","chills", "cough", "chest pain","nausea","vomiting","diarrhea", "shortness of breath"],
    "Tuberculosis": ["cough","blood tinged cough", "fever", "night sweats", "weight loss","loss of apetite"],
    "Common Cold": ["sneezing", "runny nose","nose block","fatigue", "sore throat", "cough"],
    "COVID-19": ["fever","chills", "cough","sore throat", "weakness", "loss of taste", "loss of smell", "shortness of breath","vomitting","diarrhea",],
    "Tonsillitis": ["sore throat", "difficulty swallowing","cough","ear pain", "swollen tonsils", "fever"],
    "COPD": ["chronic cough", "shortness of breath", "wheezing", "fatigue"],
    "Bronchitis": ["cough", "mucus production", "chest discomfort", "fatigue"],
    "Flu (Influenza)": ["fever", "chills", "body ache", "sore throat"],
    "Diarrhea": ["loose stools", "abdominal pain", "dehydration", "nausea"],
    "Gastroenteritis": ["vomiting", "diarrhea", "abdominal cramps", "fever"],
    "Cholera": ["watery diarrhea", "dehydration", "vomiting", "leg cramps"],
    "Food Poisoning": ["nausea", "vomiting", "diarrhea", "abdominal cramps"],
    "Peptic Ulcer": ["burning stomach pain", "bloating", "heartburn", "nausea"],
    "Acidity": ["heartburn", "bloating", "regurgitation", "chest discomfort"],
    "GERD": ["heartburn", "acid reflux", "chest pain", "difficulty swallowing"],
    "Constipation": ["hard stools", "infrequent bowel movements", "straining", "abdominal discomfort"],
    "Gallstones": ["abdominal pain", "nausea", "vomiting", "indigestion"],
    "Hepatitis": ["jaundice", "fatigue", "abdominal pain", "dark urine"],
    "Irritable Bowel Syndrome (IBS)": ["abdominal pain", "bloating", "diarrhea", "constipation"],
    "Indigestion": ["upper abdominal discomfort", "bloating", "nausea", "belching"],
    "Diabetes": ["frequent urination", "increased thirst", "weight loss", "fatigue"],
    "Hypothyroidism": ["fatigue", "weight gain", "cold intolerance", "dry skin"],
    "Hyperthyroidism": ["weight loss", "nervousness", "heat intolerance", "palpitations"],
    "Obesity": ["weight gain", "fatigue", "shortness of breath", "joint pain"],
    "Migraine": ["headache", "nausea", "sensitivity to light", "aura"],
    "Epilepsy": ["seizures", "loss of consciousness", "confusion", "staring spells"],
    "Depression": ["sadness", "loss of interest", "fatigue", "sleep problems"],
    "Anxiety Disorders": ["excessive worry", "restlessness", "sweating", "palpitations"],
    "Insomnia": ["difficulty falling asleep", "frequent awakenings", "daytime fatigue"],
    "Parkinson's Disease": ["tremor", "slowed movement", "stiffness", "balance problems"],
    "Alzheimer's Disease": ["memory loss", "confusion", "difficulty speaking", "disorientation"],
    "Autism Spectrum Disorder": ["communication difficulty", "repetitive behaviors", "social challenges"],
    "Muscular Dystrophy": ["muscle weakness", "frequent falls", "difficulty walking"],
    "Brain Tumor": ["headache", "vision problems", "nausea", "seizures"],
    "Meningitis": ["fever", "stiff neck", "headache", "sensitivity to light"],
    "Malaria": ["fever", "chills", "sweating", "headache"],
    "Dengue": ["fever", "rash", "muscle pain", "headache"],
    "Typhoid": ["fever", "abdominal pain", "weakness", "constipation"],
    "Chickenpox": ["rash", "itching", "fever", "tiredness"],
    "Measles": ["rash", "fever", "cough", "red eyes"],
    "Smallpox": ["fever", "rash", "scabs", "headache"],
    "Yellow Fever": ["fever", "jaundice", "muscle pain", "vomiting"],
    "Whooping Cough": ["severe cough", "whooping sound", "vomiting after coughing"],
    "Mumps": ["swollen glands", "fever", "headache", "muscle aches"],
    "Osteoarthritis": ["joint pain", "stiffness", "swelling", "reduced flexibility"],
    "Rheumatoid Arthritis": ["joint pain", "swelling", "fatigue", "stiffness"],
    "Gout": ["joint pain", "redness", "swelling", "heat in joints"],
    "Acne": ["pimples", "blackheads", "whiteheads", "oily skin"],
    "Eczema": ["itching", "red rash", "dry skin", "inflammation"],
    "Psoriasis": ["red patches", "scaly skin", "itching", "dryness"],
    "Glaucoma": ["eye pain", "blurred vision", "headache", "vision loss"],
    "Eye Infection": ["redness", "itching", "discharge", "pain"],
    "constipation":["hard stool","dry stool","less than three stools per week","rectum pain","no stool"],
    "stomach infection":["stomach pain or cramps","indigestion","burps","gas","fever","chills"]
}
print("✅ Rule-based dictionary ready")

✅ Rule-based dictionary ready


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # send model to device

def bert_predict(symptom_text):
    # Tokenize
    inputs = tokenizer(symptom_text,
                       return_tensors="pt",
                       truncation=True,
                       padding=True,
                       max_length=128)

    # Send inputs to same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_id = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()

    disease = le.inverse_transform([pred_id])[0]
    return disease, confidence


def rule_based_predict(symptom_text):
    text = symptom_text.lower()
    matches = []
    for disease, symptoms in disease_symptoms.items():
        score = sum(1 for s in symptoms if s in text)
        if score > 0:
            matches.append((disease, score))
    matches = sorted(matches, key=lambda x: x[1], reverse=True)
    return [m[0] for m in matches[:3]] if matches else ["No common disease match"]
print("✅ Prediction functions ready")


✅ Prediction functions ready


In [ ]:
print("Enter your symptoms (comma separated):")
user_input = input()

ml_disease, conf = bert_predict(user_input)
rule_diseases = rule_based_predict(user_input)

print("\n=== 🏥 Dhanvantari Result ===")
print(f"🤖 ML Model Prediction: {ml_disease} (confidence={conf:.2f})")
print("📋 Common Disease Matches:", rule_diseases)


Enter your symptoms (comma separated):
chest pain,chest tightness,shortness of breadth,pain in neck

=== 🏥 Dhanvantari Result ===
🤖 ML Model Prediction: Otitis Media (confidence=0.03)
📋 Common Disease Matches: ['Heart Attack (Myocardial Infarction)', 'Asthma', 'Pneumonia']


In [ ]:
!git config --global user.name "Master-Shuvam"
!git config --global user.email "23f3002495@ds.study.iitm.ac.in"  # use your GitHub email

In [ ]:
!git clone https://github.com/Master-Shuvam/Dhanvantari.git
%cd Dhanvantari

Cloning into 'Dhanvantari'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'Dhanvantari'
/content
